<a href="https://colab.research.google.com/github/bushra11812/COVID19-Face-Mask-Detection-using-deep-learning/blob/master/Occlusion_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ahmetmeleq/Face-Completion---Occlusion-Restoration-GAN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

import keras
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Flatten, Dense, Dropout, BatchNormalization, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, LeakyReLU, Conv2DTranspose, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import datetime

from keras import initializers

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=False)

In [ ]:
import gdown 
gdown.download("https://drive.google.com/uc?id={0}".format("19li26wV60jhrf8UtUhGH6xuocDqiHqPG"),"gt.pickle",quiet = False)
gdown.download("https://drive.google.com/uc?id={0}".format("179YgtbT7A0YFJsQyFULbQPgPZzdmnySA"),"occ.pickle",quiet = False)

In [ ]:
!ls


In [ ]:
pickle_in = open("occ.pickle","rb")
x = pickle.load(pickle_in)

pickle_in = open("gt.pickle","rb")
y = pickle.load(pickle_in)


In [ ]:
from skimage.transform import resize

x = resize(x, (len(x),64,64,1),anti_aliasing=False)
y= resize(y, (len(y),64,64,1),anti_aliasing=False)

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
fig = plt.figure(figsize = (6,6))
fig.add_subplot(1,2,1)
plt.imshow(x[0,:,:,0], cmap = "gray")
fig.add_subplot(1,2,2)
plt.imshow(y[0,:,:,0], cmap = "gray")


In [ ]:
def creategen():
  generator = Sequential()
  
  generator.add(Conv2D(64,(5,5),strides = (2,2), input_shape = x.shape[1:], padding = "SAME", kernel_initializer = "random_normal" ))
  generator.add(BatchNormalization())
  generator.add(ReLU())
  generator.add(Dropout(0.3))
  
  generator.add(Conv2D(128,(5,5),strides = (2,2), padding = "SAME", kernel_initializer = "random_normal" ))
  generator.add(BatchNormalization())
  generator.add(ReLU())
  generator.add(Dropout(0.3))
  
  generator.add(Conv2D(256,(5,5),strides = (2,2), padding = "SAME", kernel_initializer = "random_normal" ))
  generator.add(BatchNormalization())
  generator.add(ReLU())
  generator.add(Dropout(0.3))
  
  
  
  generator.add(Conv2DTranspose(128,(5,5),strides = (2,2), padding = "SAME", kernel_initializer = "random_normal", use_bias=False ))
  generator.add(BatchNormalization())
  generator.add(ReLU())
  #generator.add(Dropout(0.3))
  generator.add(Conv2DTranspose(64,(5,5),strides = (2,2), padding = "SAME", kernel_initializer = "random_normal", use_bias=False ))
  generator.add(BatchNormalization())
  generator.add(ReLU())
  #generator.add(Dropout(0.3))
  
  
  generator.add(Conv2DTranspose(1,(5,5),strides = (2,2), padding = "SAME", kernel_initializer = "random_normal", use_bias=False, activation = "sigmoid" ))
  
  return generator



generator = creategen()

In [ ]:
def create_disc():
  discriminator = Sequential()
  
  discriminator.add(Conv2D(64,(5,5),strides = (2,2), input_shape = x.shape[1:], padding = "SAME", kernel_initializer = "random_normal" ))
  discriminator.add(BatchNormalization())
  discriminator.add(ReLU())
  discriminator.add(Dropout(0.3))
  
  discriminator.add(Conv2D(128,(5,5),strides = (2,2), padding = "SAME", kernel_initializer = "random_normal" ))
  discriminator.add(BatchNormalization())
  discriminator.add(ReLU())
  discriminator.add(Dropout(0.3))
  
  discriminator.add(Conv2D(256,(5,5),strides = (2,2), padding = "SAME", kernel_initializer = "random_normal" ))
  discriminator.add(BatchNormalization())
  discriminator.add(ReLU())
  discriminator.add(Dropout(0.3))
  
  discriminator.add(Flatten())
  discriminator.add(Dense(1, activation = "sigmoid"))
  
  return discriminator


discriminator = create_disc()

In [ ]:
generator.summary()


In [ ]:
opt_disc = Adam(lr=0.00004)
discriminator.trainable = True
discriminator.compile(loss = "binary_crossentropy", optimizer = opt_disc)
discriminator.summary()

In [ ]:
opt_gen = Adam(lr=0.00001)
generator.compile(loss = "mean_squared_error", optimizer = opt_disc)
generator.summary()

In [ ]:
def creategan(generator,discriminator):
  gan = Sequential()
  gan.add(generator)
  
  discriminator.trainable = False
  gan.add(discriminator)
  
  return gan

gan = creategan(generator,discriminator)

In [ ]:
opt_gan = Adam(lr=0.00001)
gan.compile(loss = "binary_crossentropy", optimizer = opt_gan)
gan.summary()

In [ ]:
def train (x, y, nepoch):
    
  gen_predict = None
  
  gan_inp = x
  gan_label = np.ones(64)
  gan_predict = None
  
  
  disc_inp = None
  disc_label = np.zeros(64*2)
  disc_label[64:] = 1
  
  disc_predict = None
  
  #sess = tf.Session()
  for epoch in range(nepoch):
    for batch_ctr in range(64):
      
      gen_predict = generator.predict(gan_inp[batch_ctr*64:(batch_ctr+1)*64])
      
      if(epoch%2==0):
        
        disc_inp = gen_predict
        disc_label = np.zeros(64)
        
      else:
        disc_inp = y[batch_ctr*64:(batch_ctr+1)*64]
        disc_label = np.ones(64)
        
        
      gen_label = y[batch_ctr*64:(batch_ctr+1)*64]
      
      d_loss = discriminator.train_on_batch(disc_inp,disc_label)
      
      
      gan_loss = gan.train_on_batch(gan_inp[batch_ctr*64:(batch_ctr+1)*64],
                                   gan_label)
      
      gen_loss = generator.train_on_batch(gan_inp[batch_ctr*64:(batch_ctr+1)*64],gen_label)
  
    print("Epoch {0} Gan Loss {1} Disc Loss {2} Gen Loss {3}".format(epoch,gan_loss,d_loss,gen_loss))

In [ ]:
batch_size = 64
train(x,y,50)


In [ ]:
a = 4180
b = 4190

pred = generator.predict(x[a:b])

fig = plt.figure(figsize = (50,50))

for ctr in range(10):
  fig.add_subplot(1,10,ctr+1)
  plt.imshow(np.reshape(x[a+ctr], (64,64)), cmap = "gray")
  
fig = plt.figure(figsize = (50,50))  
for ctr in range(10):
  fig.add_subplot(1,10, ctr+1)
  plt.imshow(np.reshape(y[a+ctr], (64,64)),cmap = "gray")
  
fig = plt.figure(figsize = (50,50))  
for ctr in range(10):
  fig.add_subplot(1,10, ctr+1)
  plt.imshow(np.reshape(pred[ctr], (64,64)), cmap = "gray")